Pseudo code:
* Take a string and determine the relevant frequencies of the characters.
* Build and sort a list of tuples from lowest to highest frequencies.  
* Build the Huffman Tree by assigning a binary code to each letter, using shorter codes for the more frequent letters. (This is the heart of the Huffman algorithm.)
* Trim the Huffman Tree (remove the frequencies from the previously built tree).
* Encode the text into its compressed form.
* Decode the text from its compressed form
    

In [48]:
from collections import Counter
import heapq
import sys

class Node():
    
    def __init__(self, key, frequency):
        self.key = key
        self.frequency = frequency
        self.left = None
        self.right = None
    
    def __str__(self):
        return f"{self.key} {self.frequency}"
    
    def __lt__(self, other):
        return self.frequency < other.frequency

class HuffmanTree():
    
    def __init__(self, string):
        self.string = string
        self.frequency_table = self.get_char_frequency(string)
        self.heap = []
        self.encoder = {}
        self.decoder = {}
        self.root = None
        
    def map_codes(self, node, code):
        if node == None:
            return

        if node.key:
            self.encoder[node.key] = code
            self.decoder[code] = node.key
            return

        self.map_codes(node.left, code + "0")
        self.map_codes(node.right, code + "1")  
        
    def build_tree(self):
        for key, frequency in self.frequency_table:
            node = Node(key, frequency)
            heapq.heappush(self.heap, node)                         
        
        while len(self.heap) > 1:
            left = heapq.heappop(self.heap)
            right = heapq.heappop(self.heap)                    
            
            _new = Node(None, left.frequency + right.frequency)
            _new.left = left
            _new.right = right
            
            print("left: {} \nright:{} \nnew:{}\n".format(left, right, _new))
            
            heapq.heappush(self.heap, _new)
            
        #Root will be the node left in the priority queue
        self.root = heapq.heappop(self.heap)
        self.map_codes(self.root, "")
        
    def encode_text(self):
        return ''.join([self.encoder[char] for char in self.string]), self.root
    
    def decode_text(self, text):
        running_code = ""
        decoded_text = ""
        for bit in text:
            running_code += bit
            if running_code in self.decoder:
                decoded_char = self.decoder[running_code]
                decoded_text += decoded_char
                running_code = ""           
                
        return decoded_text
            
    def get_char_frequency(self, string):
       return sorted(Counter(string).items(), key=lambda _tuple: _tuple[1])
            
    def print_mapping(self):
        print("Encoding:")
        for key, value in self.encoder.items():
            print('key: {} value:{}'.format(key, value))

                        
a_great_sentence = "The bird is the word"
test = HuffmanTree(a_great_sentence)
test.build_tree()
test.print_mapping()
print(f'\nInput: {test.string}\nCompressed text: {test.encode_text()[0]}\n')

print ("The size of the data is: {}".format(sys.getsizeof(a_great_sentence)))
print ("The content of the data is: {}\n".format(a_great_sentence))
encoded_data, tree = test.encode_text()

print ("The size of the encoded data is: {}".format(sys.getsizeof(int(encoded_data, base=2))))
print ("The content of the encoded data is: {}\n".format(encoded_data))

decoded_data = test.decode_text(encoded_data)

print ("The size of the decoded data is: {}".format(sys.getsizeof(decoded_data)))
print ("The content of the encoded data is: {}\n".format(decoded_data))

left: T 1 
right:s 1 
new:None 2

left: o 1 
right:b 1 
new:None 2

left: w 1 
right:t 1 
new:None 2

left: h 2 
right:d 2 
new:None 4

left: None 2 
right:None 2 
new:None 4

left: e 2 
right:r 2 
new:None 4

left: None 2 
right:i 2 
new:None 4

left:   4 
right:None 4 
new:None 8

left: None 4 
right:None 4 
new:None 8

left: None 4 
right:None 8 
new:None 12

left: None 8 
right:None 12 
new:None 20

Encoding:
key: o value:0000
key: b value:0001
key: i value:001
key: h value:010
key: d value:011
key: T value:1000
key: s value:1001
key: w value:1010
key: t value:1011
key:   value:110
key: e value:1110
key: r value:1111

Input: The bird is the word
Compressed text: 1000010111011000010011111011110001100111010110101110110101000001111011

The size of the data is: 69
The content of the data is: The bird is the word

The size of the encoded data is: 36
The content of the encoded data is: 1000010111011000010011111011110001100111010110101110110101000001111011

The size of the decoded data is